In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent

import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import re
import math
from random import randint
import os
from time import sleep, localtime, strftime
from glob import glob

In [2]:
# Fake user agents for browsing
ua = UserAgent()

In [3]:
DIR_STATIC = './static'
DIR_DATA = './data'

PIA_SOCKS_USER = os.environ['PIA_SOCKS_USER']
PIA_SOCKS_PASSWORD = os.environ['PIA_SOCKS_PASSWORD']
INDEED_LOGIN = os.environ['INDEED_LOGIN']
INDEED_PASSWORD = os.environ['INDEED_PASSWORD']

# Run Behind Proxies

In [4]:
def get_driver():
    headers = {'User-Agent': ua.random}

    for key, value in enumerate(headers):
        capability_key = 'phantomjs.page.customHeaders.{}'.format(key)
        webdriver.DesiredCapabilities.PHANTOMJS[capability_key] = value

    service_args = ['--proxy=proxy-nl.privateinternetaccess.com:1080', 
                    '--proxy-type=socks5', 
                    '--proxy-auth={0}:{1}'.format(PIA_SOCKS_USER, PIA_SOCKS_PASSWORD),
                    '--web-security=false',
                    '--ignore-ssl-errors=true']

    driver = webdriver.PhantomJS('phantomjs', service_args=service_args)
    return driver

In [5]:
driver = get_driver()

In [6]:
def enter_field_text(field_id, field_input, key_press, save_pic=False):
    field = driver.find_element_by_id(field_id)
    field.send_keys(field_input)
    field.send_keys(key_press)
    
    if save_pic:
        driver.set_window_size(1080,720)
        driver.save_screenshot('{0}/{1}.png'.format(DIR_DATA, field_id))
        
def login_indeed():
    # go to login page
    driver.get('https://secure.indeed.com/account/login?service=rex&hl=en_US&co=US&cfb=2&continue=http%3A%2F%2Fwww.indeed.com%2Fresumes')
    
    enter_field_text('signin_email', INDEED_LOGIN, Keys.TAB, True)
    enter_field_text('signin_password', INDEED_PASSWORD, Keys.ENTER, True)

In [7]:
login_indeed()

# Get Number of Pages for Title

In [8]:
def get_number_results(title):
    url = 'http://www.indeed.com/resumes?q={0}&co=US'.format(title)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # get number of results for query
    result_count = soup.find(attrs={'id': 'result_count'})

    try:
        # grab number only
        num = (re.findall(r'''([0-9,]+)''', str(result_count)))[0]
        num = int(num.replace(',', ''))
        return num
    except:
        return 0

In [9]:
title = 'software'
get_number_results(title)

6082011

# Make List of Links

In [10]:
def get_links(title):
    list_links = []
    list_links.append('http://www.indeed.com/resumes?q={0}&co=US'.format(title))

    for start_page in range(50, get_number_results(title), 50):
        list_links.append('http://www.indeed.com/resumes?q={0}&co=US&start={1}'.format(title, str(start_page)))

    return list_links

In [ ]:
def get_users(url):
    print("Entered 'get_users with {0}".format(url))
    temp = []
    ct = 0
    while len(temp) == 0:
        if ct >= 5: break
#         try:
        driver.get(url)
        ct += 1
        ### SLEEP
#             sleep(randint(2,8))
        ### SLEEP
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # find users
        list_of_people = soup.find_all(attrs={'class':'clickable_resume_card'})

        print(list_of_people)
        # make list of users found
        temp = [re.search(r'''(\/.+)\?''', person.get('onclick')).group() for person in list_of_people]
        
#         except:
#             continue

    return temp

In [ ]:
get_users('https://www.indeed.com/resumes?q=software&co=US')

Entered 'get_users with https://www.indeed.com/resumes?q=software&co=US
[]
[]


In [12]:
def scrape_users(title):
    # cleanup previous file versions before starting due to append
    f = '{0}.csv'.format(title)
    if os.path.exists(f): os.remove(f)
    if os.path.exists(f): os.remove('failed.csv')
    
    ttl = 0 # count
    ct = 0
    # loop through pages
    for name in get_links(title):
            # make list of users found
            temp = get_users(name)
            print("Returned from 'get_users' with {0} results".format(len(temp)))

            # show what ip is being used
            before = ttl
            ttl += len(temp)
            print('List: {0}'.format(ttl))
            
            if before != ttl:
                # write list of names found to file
                title = title.replace(' ', '_')
                with open('data/{0}.csv'.format(title), 'a') as outfile:
                    for name in temp:
                        outfile.write('{0}, \n'.format(name))
                ct += 1
                if ct % 3 == 0:
                    driver.delete_all_cookies()
            else:
                print('#'*80)
                print('No new results')
                print('#'*80)
                failed_page(title, ttl)
                driver.dispose()
                break

In [13]:
def failed_page(title, start_page):
    # if scrape failes, output point where it failed
    with open('failed.csv', 'a') as outfile:
        outfile.write(('title:{0}, page_start:{1}, \n').format(title, start_page))

# Start the Scraper

In [14]:
scrape_users(title)

Entered 'get_users with http://www.indeed.com/resumes?q=software&co=US
Returned from 'get_users' with 0 results
List: 0
################################################################################
No new results
################################################################################


AttributeError: 'WebDriver' object has no attribute 'dispose'